In [1]:

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [2]:
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
data = load_breast_cancer()
X, y = data.data, data.target

In [4]:
print(X.shape, y.shape)

(569, 30) (569,)


In [5]:
print(data['feature_names'])

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_tensor = torch.from_numpy(X_train_scaled).float().to(device) # Move tensors to the same device as the model
X_test_tensor = torch.from_numpy(X_test_scaled).float().to(device)
y_train_tensor = torch.from_numpy(y_train).long().to(device)
y_test_tensor = torch.from_numpy(y_test).long().to(device)

In [9]:
class BrestCancerModel(nn.Module):
    def __init__(self, input_dim, hidden1_dim, hidden2_dim, output_dim):
        super(BrestCancerModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1_dim)
        self.fc2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.fc3 = nn.Linear(hidden2_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # cross-entropy does categorization 
        return x

In [10]:
input_dim = X_train_tensor.shape[1] # 30
hidden1_dim = 32
hidden2_dim = 16
output_dim = 2

In [11]:
model = BrestCancerModel(input_dim, hidden1_dim, hidden2_dim, output_dim)
model.to(device)
print(model.eval())

BrestCancerModel(
  (fc1): Linear(in_features=30, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)


In [12]:
criterian = nn.CrossEntropyLoss()
optimize = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

epoch = 100
start_time = time.time()

for i in range(epoch+1):

    output = model(X_train_tensor)
    loss = criterian(output, y_train_tensor)
    
    optimize.zero_grad()
    loss.backward()
    optimize.step()
    
    if (i) % 10 == 0 :
        print(f'Epoch: {i}, Loss: {loss.item()}')
        
end_time = time.time()

print(f'Total Time: {end_time - start_time}')

Epoch: 0, Loss: 0.6809134483337402
Epoch: 10, Loss: 0.6102492809295654
Epoch: 20, Loss: 0.5248410701751709
Epoch: 30, Loss: 0.4318874776363373
Epoch: 40, Loss: 0.33804720640182495
Epoch: 50, Loss: 0.25083813071250916
Epoch: 60, Loss: 0.18144051730632782
Epoch: 70, Loss: 0.1354677379131317
Epoch: 80, Loss: 0.10754945874214172
Epoch: 90, Loss: 0.09018968045711517
Epoch: 100, Loss: 0.07873162627220154
Total Time: 0.30585598945617676


In [14]:
with torch.no_grad():
    y_train_pred = model(X_train_tensor)
    y_test_pred = model(X_test_tensor)
    y_train_pred = torch.argmax(y_train_pred, dim=1).cpu().numpy()
    y_test_pred = torch.argmax(y_test_pred, dim=1).cpu().numpy()

In [15]:
print("Accuracy Score on Training Data: ", accuracy_score(y_train, y_train_pred))
print(classification_report(y_train, y_train_pred))
print("Accuracy Score on Testing Data: ", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


Accuracy Score on Training Data:  0.9824175824175824
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       170
           1       0.99      0.99      0.99       285

    accuracy                           0.98       455
   macro avg       0.98      0.98      0.98       455
weighted avg       0.98      0.98      0.98       455

Accuracy Score on Testing Data:  0.9473684210526315
              precision    recall  f1-score   support

           0       0.89      0.98      0.93        42
           1       0.99      0.93      0.96        72

    accuracy                           0.95       114
   macro avg       0.94      0.95      0.94       114
weighted avg       0.95      0.95      0.95       114



In [21]:
sample = X_test_scaled[0].reshape(1, -1)
print(sample)
sample = torch.from_numpy(sample).float().to(device)
with torch.no_grad():
	output = model(sample) # 0 : malignant, 1 : benign
	print(output)
	predicted_class = torch.argmax(output, dim=1).item()
	print("Predicted Class:", predicted_class)

[[ 1.56851278  2.16401585  1.74286587  1.62076364 -0.26537607  1.91593165
   1.09245512  1.69889576  0.30958918 -0.06988421  1.6582331  -0.03455846
   2.19927944  1.6097947  -0.2323368   1.19871549  0.18795672  0.55981664
  -0.16207336  0.52383207  1.86216011  1.70820774  2.17062246  1.91370109
  -0.18774006  1.74427462  0.68856004  1.20214432 -0.14043064  0.90036171]]
tensor([[ 6.7396, -2.9284]], device='cuda:0')
Predicted Class: 0
